In [20]:
import csv
import math
import re
import pandas as pd
import requests
import numpy as np
import io

Dataset Location

The Dataset cannot be pushed to Github. Github has decreased the allowed file size so the dataset will have to be downloaded manually.

The url is: https://data.cms.gov/provider-data/sites/default/files/resources/69a75aa9d3dc1aed6b881725cf0ddc12_1671768321/DAC_NationalDownloadableFile.csv

The Data Dictionary for this section of datasets: https://data.cms.gov/provider-data/sites/default/files/data_dictionaries/physician/DOC_Data_Dictionary.pdf

The webpage for this section of datasets: https://data.cms.gov/provider-data/topics/doctors-clinicians


In [5]:
df = pd.read_csv('NationalDocData/DAC_NationalDownloadableFile.csv', encoding='windows-1252', dtype='string')

(TODO) Filtering and Cleaning the Dataset

Note: Since this is for Physicians, we will need to filter out non-physicians, but keep in mind that some physicians will also have a secondary degree listed under their specialities, so the filtering needs to be a bit smarter

In [6]:
# NOTE: Optimize This
addresses = df[['adr_ln_1', 'cty', 'st', 'zip','pri_spec']]
addresses.drop_duplicates()
addresses.dropna()
addresses = addresses[addresses['adr_ln_1'].str.strip().astype(bool)]
addresses = addresses[addresses['cty'].str.strip().astype(bool)]
addresses = addresses[addresses['st'].str.strip().astype(bool)]
addresses = addresses[addresses['zip'].str.strip().astype(bool)]
addresses = addresses[addresses['pri_spec'].str.strip().astype(bool)]
addresses['pri_spec'] = addresses['pri_spec'].astype(str)
print(addresses.head())
print(addresses.shape[0])

             adr_ln_1             cty  st        zip                pri_spec
0     1582 N BROAD ST        TAZEWELL  TN  378794352      NURSE PRACTITIONER
1   5632 ANNAPOLIS RD     BLADENSBURG  MD  207102213      NURSE PRACTITIONER
2    611 E CARLSON ST        CHEYENNE  WY  820094335       INTERNAL MEDICINE
3         30 RIVER ST      JIM THORPE  PA  182292313            CHIROPRACTIC
4  20501 VENTURA BLVD  WOODLAND HILLS  CA  913640847  CLINICAL SOCIAL WORKER
2472219


In [7]:
#Printing Unique Primary Specialites to Filter From
print(addresses['pri_spec'].unique())

['NURSE PRACTITIONER' 'INTERNAL MEDICINE' 'CHIROPRACTIC'
 'CLINICAL SOCIAL WORKER' 'OPTOMETRY' 'ANESTHESIOLOGY' 'FAMILY PRACTICE'
 'NEUROLOGY' 'CLINICAL PSYCHOLOGIST' 'DERMATOLOGY' 'PHYSICAL THERAPY'
 'OPHTHALMOLOGY' 'ORTHOPEDIC SURGERY' 'HOSPITALIST' 'PSYCHIATRY'
 'ORAL SURGERY' 'GENERAL PRACTICE' 'PLASTIC AND RECONSTRUCTIVE SURGERY'
 'PHYSICAL MEDICINE AND REHABILITATION' 'GASTROENTEROLOGY' 'UROLOGY'
 'PULMONARY DISEASE' 'CERTIFIED REGISTERED NURSE ANESTHETIST (CRNA)'
 'REGISTERED DIETITIAN OR NUTRITION PROFESSIONAL' 'PODIATRY'
 'ENDOCRINOLOGY' 'CARDIOVASCULAR DISEASE (CARDIOLOGY)'
 'MAXILLOFACIAL SURGERY' 'QUALIFIED AUDIOLOGIST' 'OBSTETRICS/GYNECOLOGY'
 'PHYSICIAN ASSISTANT' 'PEDIATRIC MEDICINE' 'NEUROSURGERY' 'RHEUMATOLOGY'
 'QUALIFIED SPEECH LANGUAGE PATHOLOGIST' 'INTERVENTIONAL PAIN MANAGEMENT'
 'OCCUPATIONAL THERAPY' 'EMERGENCY MEDICINE' 'PAIN MANAGEMENT'
 'INTERVENTIONAL CARDIOLOGY' 'NEPHROLOGY' 'INFECTIOUS DISEASE'
 'GENERAL SURGERY' 'GERIATRIC MEDICINE' 'ALLERGY/IMMUNOLOGY'
 

In [8]:
batches = math.ceil(addresses.shape[0] / 10000)
print(batches)

248


Steps to Get Geolocations of Addresses in Batch

Create a loop that generates a CSV for each 247 entries
Send the generated CSV to the Census GOV Geolocation API for processing
Recieve the response and append it to a local CSV
Rinse and repeat until all addresses are converted to geolocations (lat. and long.) to use with a map

In [102]:
URL = 'https://geocoding.geo.census.gov/geocoder/locations/addressbatch'
with open('NationalDocData/test_addresses.csv', 'rb') as input_file:
    geo = requests.post(files={'addressFile': input_file}, url=URL, data={'benchmark':'Public_AR_Current'})

In [103]:
print(geo.content)

b'"000001","8 E View Dr, Whitehouse Station, NJ, 08889","Match","Exact","8 E VIEW DR, WHITEHOUSE STATION, NJ, 08889","-74.76707085499999,40.611750001000075","627779463","R"\n'


In [104]:
print(type(geo))

<class 'requests.models.Response'>


In [105]:
#data = need to change response to text and separate values

df = pd.read_csv(io.StringIO(geo.text), sep=',', header=None, quoting=csv.QUOTE_ALL, index_col= False)

with pd.option_context(
    'display.width', None, 
    'display.max_columns', None,
    'display.max_colwidth', -1,
    'display.colheader_justify', 'left'):
    print(df)

   0 1                                           2      3       \
0  1  8 E View Dr, Whitehouse Station, NJ, 08889  Match  Exact   

  4                                            \
0  8 E VIEW DR, WHITEHOUSE STATION, NJ, 08889   

  5                                       6         7   
0  -74.76707085499999,40.611750001000075  627779463  R  


In [106]:
#find latitude and longitude column
df.reset_index()

,index,0,1,2,3,4,5,6,7
0,0,1,"8 E View Dr, Whitehouse Station, NJ, 08889",Match,Exact,"8 E VIEW DR, WHITEHOUSE STATION, NJ, 08889","-74.76707085499999,40.611750001000075",627779463,R


If you have a DataFrame with only one row, then access the first (only) row as a Series using iloc, and then the value using the column name:

In [107]:
coord = df.iloc[0][5]
lat, lon = coord.split(',')

In [108]:
lat

'-74.76707085499999'

In [109]:
lon

'40.611750001000075'